In [1]:
#import all the libraries 
import matplotlib.pyplot as plt
import pandas      as pd
import numpy       as np

In [2]:
#read the csv.file
data = pd.read_csv("h1b_kaggle.csv",index_col=0)

C:\Users\eruba\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
#no of rows and columns in dataset
print('No of rows and columns: ',data.shape)
print('\n Total no of entry in each column: \n', data.count())

#no of missing value
print('\n No of missing values: ')
for col in cdf.columns:
    print (col,cdf[col].isnull().sum())

In [3]:
#made a deep copy of dataset
cdf = data.copy()

In [4]:
#drop rows with missing values [each row with total 7 columns of NAN values]
cdf = cdf.dropna(axis='rows', thresh=7)    

In [ ]:
# Feature 1: CASE_STATUS
cdf['CASE_STATUS'].value_counts()

In [5]:
# merge 'CERTIFIED-WITHDRAWN' value with 'CERTIFIED' 
cdf['CASE_STATUS'] = np.where(cdf['CASE_STATUS']=='CERTIFIED-WITHDRAWN','CERTIFIED',cdf['CASE_STATUS'])

# drop rows with 'WITHDRAWN' value 
indexNames = cdf[cdf['CASE_STATUS']=='WITHDRAWN'].index
cdf.drop(indexNames , inplace=True)

In [6]:
#replace 'CASE_STATUS' labels with numeric values from '1-6'
cdf['CASE_STATUS'] = cdf['CASE_STATUS'].replace({'CERTIFIED': 1,'DENIED':2,'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED':3,'REJECTED':4,'INVALIDATED':5}).astype(int)

In [7]:
# Feature 2: EMPLOYER_NAME
#remove special characters to uniform format to get a better understanding of the dataset and enforce consistency. 
cdf['EMPLOYER_NAME']=cdf['EMPLOYER_NAME'].str.lower().replace("&QUOT;","")    
cdf['EMPLOYER_NAME']=cdf['EMPLOYER_NAME'].str.replace("[.,)\"'&(+/]","")   

In [8]:
# Feature 3: SOC_NAME
#remove special characters to uniform format
cdf['SOC_NAME']=cdf['SOC_NAME'].str.lower().replace("<FONT>|</FONT>|^[0-9]$","")

#drop ambiguous values
a = ['13-2011.01','15-1121','15-1132','15-1199.01','17-2051','27-3031']
cdf = cdf[~cdf['SOC_NAME'].isin(a)]

In [9]:
# Feature 4: JOB_TITLE
#remove special characters to uniform format
cdf['JOB_TITLE']=cdf['JOB_TITLE'].str.lower().replace("[.,)\-]","")

In [ ]:
cdf['JOB_TITLE'].describe()

In [10]:
# Feature 5: FULL_TIME_POSITION
#replace 'Y' and 'N' label of 'FULL_TIME_POSITION' respectively with '1' and '0'
cdf['FULL_TIME_POSITION']=cdf['FULL_TIME_POSITION'].replace({'Y': 1, 'N': 0})

In [11]:
# Feature 7: YEAR
# convert year as integer type value
cdf['YEAR'] = cdf['YEAR'].astype(int)

In [12]:
# Feature 8: WORKSITE
#select state as employer location
cdf['WORKSITE']=cdf['WORKSITE'].str.split(',',1).str[1]

In [13]:
# Feature 8: lattitude and longitutde
#drop lattitude and longitutde
cdf.drop('lon',inplace=True,axis=1)
cdf.drop('lat',inplace=True,axis=1)

# Statistical Analysis 

In [ ]:
#Top 10 companies with most application

comp_counts = cdf['EMPLOYER_NAME'].value_counts().head(10)
print(comp_counts)
ax = comp_counts.plot.barh()
ax.invert_yaxis()
ax.set_title('Top 10 Company with most applications')
ax.set_xlabel('Frequency')
ax.set_ylabel('Company Name')
plt.show()

In [ ]:
#Total number of case file in each year
cdf.groupby(['YEAR', 'CASE_STATUS']).size().unstack().plot(kind='bar', stacked=True)

In [ ]:
#ratio of application outcomes for each year
pd.crosstab(cdf['YEAR'],cdf['CASE_STATUS']).apply(lambda r: r/r.sum()*100, axis=1)

In [14]:
# min, max and median wage 
median_wage = np.nanmedian(cdf[['PREVAILING_WAGE']])
print('Median wage: ', median_wage)
cdf["PREVAILING_WAGE"].describe()

Median wage:  65000.0


count    2.912587e+06
mean     1.428912e+05
std      5.282317e+06
min      0.000000e+00
25%      5.441300e+04
50%      6.500000e+04
75%      8.143200e+04
max      6.997607e+09
Name: PREVAILING_WAGE, dtype: float64

In [15]:
#detect outlier in PREVAILING_WAGE
q1 = cdf["PREVAILING_WAGE"].quantile(0.25)
q3 = cdf["PREVAILING_WAGE"].quantile(0.75)
IQR = q3 - q1
outliers = ((cdf["PREVAILING_WAGE"] < (q1 - 1.5 * IQR)) | (cdf["PREVAILING_WAGE"] > (q3 + 1.5 * IQR))).sum()
print('No of outliers: ', outliers)

No of outliers:  113489


In [17]:
# replace outlier with median value
cdf["PREVAILING_WAGE"] = cdf["PREVAILING_WAGE"].mask(((cdf["PREVAILING_WAGE"] < (q1 - 1.5 * IQR)) | (cdf["PREVAILING_WAGE"] > (q3 + 1.5 * IQR))), median_wage)

In [ ]:
#top 10 preffered type of job 
job_counts = cdf['SOC_NAME'].value_counts().head(10)
print(job_counts)

#plot the findings
ax = job_counts.plot.barh()
ax.invert_yaxis()
ax.set_title('Top 10 prefferable job title')
ax.set_xlabel('Frequency')
ax.set_ylabel('Job Title')
plt.show()

In [ ]:
#top 5 cities with most job opportunity

cdf['WORKSITE'].value_counts().head(5).plot(kind='pie',startangle=90, autopct='%.1f%%')
plt.title('Top 5 cities with most job opportunity')

In [ ]:
#visa application success ratio based on job status
table = pd.crosstab(cdf['CASE_STATUS'], cdf['FULL_TIME_POSITION'],
margins=True)
table = table.sort_values(by='All', ascending=False)
table.drop('All', inplace=True)
table.drop('All', axis='columns', inplace=True)
ax = table.plot.barh()
ax.invert_yaxis()
ax.set_title('Application status based on job status')
ax.set_xlabel('Frequency')
ax.set_ylabel('Application Status')
plt.show()